In [37]:
import polars as pl
import json
from requests import get
import re

In [14]:
from typing import TypedDict

**EXERCICE** Récupérer le fichier json à l'adresse "https://raw.githubusercontent.com/VPerrollaz/immobilier/refs/heads/master/donnees/brute.json".
Puis le desérialiser.

In [3]:
ADRESSE_JSON = "https://raw.githubusercontent.com/VPerrollaz/immobilier/refs/heads/master/donnees/brute.json"

In [7]:
requete = get(ADRESSE_JSON)
assert requete.status_code == 200

In [8]:
contenu = requete.text

In [15]:
# TypedDict permet au typechecker de constater les bogues
# de clefs ou de type de valeur
class Annonce(TypedDict):
    id: str
    genre: str
    prix: str
    desc: str
    lien: str

In [11]:
deserialises: list[Annonce] = list()
for ligne in contenu.splitlines():
    deserialises.append(json.loads(ligne))

In [12]:
len(deserialises)

1818

In [13]:
deserialises[0]

{'id': 'annonce-138905473-376235',
 'genre': 'Appartement',
 'prix': '374 400 €',
 'pcs': '3 p 2 ch 90 m²',
 'desc': "Appartement type 3 - TOURS CATHÉDRALE TOURS CATHÉDRALE: Ravissant appartement type 3 d'environ 90 m² dans résidence de standing (~50 lots) avec cave et parking. Entrée,...\nÇa m'intéresse",
 'lien': 'https://www.seloger.com/annonces/achat/appartement/tours-37/cathedrale/138905473.htm'}

**EXERCICE** Extraire un dataframe `polars` du json.

On consultera les documentations de `polars.read_json` et `polars.DataFrame`

In [16]:
brut = pl.DataFrame(data=deserialises)

In [17]:
brut.describe()

statistic,id,genre,prix,pcs,desc,lien
str,str,str,str,str,str,str
"""count""","""1818""","""1818""","""1818""","""1818""","""1818""","""1818"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null,null
"""std""",null,null,null,null,null,null
"""min""","""annonce-100241563-317111""","""Appartement""","""1 030 000 €""","""""","""""TOURS GRAMMONT"" Dernier étage…","""https://www.bellesdemeures.com…"
"""25%""",null,null,null,null,null,null
"""50%""",null,null,null,null,null,null
"""75%""",null,null,null,null,null,null
"""max""","""annonce-98789327-311359""","""Terrain""","""€""","""92 m²""","""ÉLÉGANTE MAISON BOURGEOISE, po…","""https://www.selogerneuf.com/an…"


L'objectif final est d'obtenir un dataset sous forme _tidy_ c'est à dire chaque colonne représente une variable explicative numérique.

**EXERCICE** supprimer les doublons, et les lignes non pertinentes en général.

In [19]:
# Y  a t il des lignes dupliquées?
brut.is_duplicated().describe()

statistic,value
str,f64
"""count""",1818.0
"""null_count""",0.0
"""mean""",0.022002
"""min""",0.0
"""max""",1.0


In [20]:
brut = brut.unique()

In [21]:
brut.is_duplicated().describe()

statistic,value
str,f64
"""count""",1798.0
"""null_count""",0.0
"""mean""",0.0
"""min""",0.0
"""max""",0.0


In [22]:
brut.columns

['id', 'genre', 'prix', 'pcs', 'desc', 'lien']

**REMARQUE** la colonne `id` a permis de supprimer les doublons de manière sûre, elle pourra être abandonner maintenant.

In [23]:
brut["genre"]

genre
str
"""Appartement"""
"""Maison / Villa"""
"""Appartement"""
"""Appartement"""
"""Maison / Villa"""
…
"""Appartement"""
"""Appartement"""
"""Maison / Villa"""


Comme on a une variable en fait catégorielle, on veut voir les catégories présentes.

In [24]:
brut["genre"].unique()

genre
str
"""Bâtiment"""
"""Bureau"""
"""Projet de construction"""
"""Local commercial"""
"""Parking"""
…
"""Appartement"""
"""Boutique"""
"""Appartement neuf"""


On voit qu'il y a des catégories qui ne correspondent pas à des habitations!

In [27]:
brut.group_by("genre").len()

genre,len
str,u32
"""Maison / Villa""",376
"""Hôtel particulier""",1
"""Boutique""",14
"""Maison / Villa neuve""",16
"""Appartement""",1037
…,…
"""Terrain""",3
"""Local commercial""",30
"""Immeuble""",32


In [32]:
list(brut["genre"].unique())

['Bureau',
 'Hôtel particulier',
 'Projet de construction',
 'Local commercial',
 'Divers',
 'Château',
 'Parking',
 'Appartement neuf',
 'Boutique',
 'Loft/Atelier/Surface',
 'Appartement',
 'Terrain',
 'Immeuble',
 'Maison / Villa',
 'Maison / Villa neuve',
 'Bâtiment']

**EXERCICE** On ne veut conserver que les annonces correspondant à un genre:
- Appartement neuf
- Appartement
- Maison / Villa
- Maison / Villa neuve

On commencera par regarder la documentation de `DataFrame.filter`.

In [34]:
genre_valides = {
    'Appartement neuf', 
    'Appartement', 
    'Maison / Villa', 
    'Maison / Villa neuve'
}
brut = brut.filter(pl.col("genre").is_in(genre_valides)).drop("id")

In [35]:
brut.describe()

statistic,genre,prix,pcs,desc,lien
str,str,str,str,str,str
"""count""","""1633""","""1633""","""1633""","""1633""","""1633"""
"""null_count""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null
"""std""",null,null,null,null,null
"""min""","""Appartement""","""1 030 000 €""","""""","""""TOURS GRAMMONT"" Dernier étage…","""https://www.bellesdemeures.com…"
"""25%""",null,null,null,null,null
"""50%""",null,null,null,null,null
"""75%""",null,null,null,null,null
"""max""","""Maison / Villa neuve""","""€""","""9 p 8 ch 280 m²""","""ÉLÉGANTE MAISON BOURGEOISE, po…","""https://www.selogerneuf.com/an…"


**EXERCICE** La colonne suivante est `prix`, comme c'est la valeur cible, il faut être sûr de ne garder que les annonces qui ont un prix valide.

On commencera par construire une regex qui matche un prix valide.

In [36]:
brut["prix"]

prix
str
"""236 250 €"""
"""629 000 €"""
"""146 400 €"""
"""94 000 €"""
"""1 924 000 €"""
…
"""203 300 €"""
"""425 000 €"""
"""370 000 € HH"""


In [55]:
prix_valide = "^[0-9 ]+ €.*$"

In [56]:
re.match(prix_valide, "1 924 000 €")

<re.Match object; span=(0, 11), match='1 924 000 €'>

In [57]:
re.match(prix_valide, "370 000 € HH")

<re.Match object; span=(0, 12), match='370 000 € HH'>

In [58]:
brut = brut.filter(pl.col("prix").str.count_matches(prix_valide) > 0)

In [52]:
brut.describe()

statistic,genre,prix,pcs,desc,lien
str,str,str,str,str,str
"""count""","""1632""","""1632""","""1632""","""1632""","""1632"""
"""null_count""","""0""","""0""","""0""","""0""","""0"""
"""mean""",null,null,null,null,null
"""std""",null,null,null,null,null
"""min""","""Appartement""","""1 030 000 €""","""1 p 1 ch 29 m²""","""""TOURS GRAMMONT"" Dernier étage…","""https://www.bellesdemeures.com…"
"""25%""",null,null,null,null,null
"""50%""",null,null,null,null,null
"""75%""",null,null,null,null,null
"""max""","""Maison / Villa neuve""","""992 000 €""","""9 p 8 ch 280 m²""","""ÉLÉGANTE MAISON BOURGEOISE, po…","""https://www.selogerneuf.com/an…"


**EXERCICE** Explorer la variable `pcs` puis filtrer ce qui est pertinent.

In [53]:
brut["pcs"].sample(10)

pcs
str
"""5 p 3 ch 150 m²"""
"""5 p 3 ch 88,2 m²"""
"""2 p 1 ch 42 m²"""
"""2 p 1 ch 43 m²"""
"""2 p 1 ch 36 m²"""
"""2 p 1 ch 30 m²"""
"""3 p 2 ch 69 m²"""
"""8 p 5 ch 196,44 m²"""
"""2 p 1 ch 50,67 m²"""


In [68]:
motif_surface = "(([0-9]+)|([0-9]+,[0-9]+)) m²"

In [69]:
re.findall(motif_surface, "5 p 3 ch 150 m²")

[('150', '150', '')]

In [70]:
re.findall(motif_surface, "5 p 3 ch 88,2 m²")

[('88,2', '', '88,2')]

In [71]:
re.findall(motif_surface, "8 p 5 ch 196,44 m²")

[('196,44', '', '196,44')]

In [72]:
brut = brut.filter(pl.col("pcs").str.find(motif_surface).is_not_null())

In [73]:
motif_pieces = "[0-9]+ p"

In [76]:
brut = brut.filter(pl.col("pcs").str.find(motif_pieces).is_not_null())

In [80]:
brut.shape

(1625, 5)

In [77]:
motif_chambre = "[0-9]+ ch"

In [79]:
brut.filter(pl.col("pcs").str.find(motif_chambre).is_not_null()).shape

(1450, 5)

**REMARQUE** il y a 175 annonces sans renseignement concernant le nombre de chambres.
On a trois possibilités:
- On filtre pour ne garder que les annonces avec cette information
- On ne filtre pas et on abandonne cette variable explicative
- On ne filtre pas et on cherche à entrer une valeur pertinente (ou Null) lorsque l'information manque (on peut par exemple partir de l'hypothèse qu'il n'y a qu'une seule chambre)

In [82]:
brut.filter(pl.col("pcs").str.find(motif_chambre).is_null()).sample(15)

genre,prix,pcs,desc,lien
str,str,str,str,str
"""Maison / Villa neuve""","""320 000 €""","""4 p 84,4 m²""","""Maisons de ville 3 chambres av…","""https://www.selogerneuf.com/an…"
"""Appartement""","""35 000 €""","""1 p 33 m² 1 asc""","""En exclusivité chez FONCIA. Ré…","""https://www.seloger.com/annonc…"
"""Appartement neuf""","""129 600 €""","""2 p 40 m²""","""LE NOUVEAU CŒUR DE VIE A TOURS…","""https://www.selogerneuf.com/an…"
"""Appartement""","""82 500 €""","""1 p 36,59 m² 1 asc""","""Une EXCLUSIVITÉ CITYA pour un …","""https://www.seloger.com/annonc…"
"""Appartement neuf""","""188 500 €""","""2 p 54,1 m² 1 asc""","""LANCEMENT COMMERCIAL ! Renseig…","""https://www.selogerneuf.com/an…"
…,…,…,…,…
"""Appartement""","""95 500 €""","""1 p 35 m² 5 etg""","""** EXCEPTIONNEL FRAIS D AGENCE…","""https://www.seloger.com/annonc…"
"""Appartement neuf""","""271 000 €""","""4 p 89 m² tess""","""Au nord du centre-ville de Tou…","""https://www.selogerneuf.com/an…"
"""Maison / Villa neuve""","""478 000 €""","""4 p 113 m²""","""Face à la Loire, Panoramic est…","""https://www.selogerneuf.com/an…"


On ne voit pas forcément de stratégie d'input évident on va prendre la deuxième option.

**CONCLUSION** on ne voit pas de filtre évident sur les deux dernières colonnes, on s'arrête ici.

**EXERCICE** Produire un DataFrame avec les variables explicatives
- nombre de pièce (int)
- surface (float)
- Maison (Vrai ou Faux (appartement))
- Neuf (Vrai ou Faux)
- Prix (int)

On commencera en regardant la documentation de `DataFrame.select`.

In [105]:
motif_prix = "([0-9 ]+) €.*"
motif_surface = "(([0-9]+)|([0-9]+,[0-9]*)) m²"
motif_pieces = "([0-9]+) p"
motif_maison = "Maison"
motif_neuf = "(neuf|neuve)"

In [111]:
dataset = brut.select(
    pl.col("prix").str.extract(motif_prix).str.replace_all(" ", "").cast(int),
    pl.col("pcs").str.extract(motif_surface).str.replace(",", ".").cast(float).alias("surface"),
    pl.col("pcs").str.extract(motif_pieces).cast(int).alias("Nombre pieces"),
    (pl.col("genre").str.count_matches(motif_maison) > 0).alias("Maison"),
    (pl.col("genre").str.count_matches(motif_neuf) > 0).alias("Neuf"),
)

In [115]:
# vérification Maison Neuf
print(dataset.group_by("Maison").len())
print(dataset.group_by("Neuf").len())
print(brut.group_by("genre").len())

shape: (2, 2)
┌────────┬──────┐
│ Maison ┆ len  │
│ ---    ┆ ---  │
│ bool   ┆ u32  │
╞════════╪══════╡
│ false  ┆ 1234 │
│ true   ┆ 391  │
└────────┴──────┘
shape: (2, 2)
┌───────┬──────┐
│ Neuf  ┆ len  │
│ ---   ┆ ---  │
│ bool  ┆ u32  │
╞═══════╪══════╡
│ true  ┆ 219  │
│ false ┆ 1406 │
└───────┴──────┘
shape: (4, 2)
┌──────────────────────┬──────┐
│ genre                ┆ len  │
│ ---                  ┆ ---  │
│ str                  ┆ u32  │
╞══════════════════════╪══════╡
│ Maison / Villa neuve ┆ 16   │
│ Appartement neuf     ┆ 203  │
│ Appartement          ┆ 1031 │
│ Maison / Villa       ┆ 375  │
└──────────────────────┴──────┘
